In [1]:
library(effsize)

In [2]:
# load and set variables
sample_type <- 'blood serum'
huna_init <- read.csv('/Users/morris/Box/Hall_Lab/Projects/HUNA/MTBLS424/MTBLS424_init.csv',
                      header=TRUE, check.names=FALSE)
nrow <- nrow(huna_init)
print('ASSIGNED VARIABLES:')
for (i in 1:nrow){
    assign(toString(huna_init$type[i]), toString(huna_init$path[i]))
    print(toString(huna_init$type[i]))
}

[1] "ASSIGNED VARIABLES:"
[1] "setwd_path"
[1] "data_set_root_dir"
[1] "path_to_fid"
[1] "norm_bin_500_path"
[1] "norm_bin_5000_path"
[1] "bin500_for_sig_tests"
[1] "figure_path"
[1] "sample_labels_header"
[1] "stat_results_data_path"
[1] "BonSig_Cohen_results_data_path"


In [3]:
nmr_data <- read.csv(stat_results_data_path, header=TRUE, check.names=FALSE)
nmr_data$Spectral_Bin <- substring(nmr_data$Spectral_Bin, 2)

In [4]:
nmr_power <- nmr_data[nmr_data$Power_CohenVal_Threshold == TRUE,]
print(paste('Number of spectral bins with >=80% Power: ',nrow(nmr_power)))
data <- nmr_power[nmr_power$WRS_Test_Sig_Bonferroni == TRUE,]
print(paste('Number of spectral bins with Bonferroni corrected significance: '
            ,nrow(data)))

[1] "Number of spectral bins with >=80% Power:  448"
[1] "Number of spectral bins with Bonferroni corrected significance:  127"


In [5]:
# urine ROI
if (sample_type == 'urine'){
    all_ROI <- read.csv('/Users/morris/Box/Hall_Lab/Projects/MTBLS_explore/MTBLS1/input/All_urine_ROI.csv',
                    header=TRUE, check.names=FALSE)
}
if (sample_type == 'blood serum'){
    all_ROI <- read.csv('/Users/morris/Box/Hall_Lab/Projects/MTBLS_explore/MTBLS1/input/All_Blood_ROI.csv',
                header=TRUE, check.names=FALSE)
}

In [6]:
filtered_ROI <- all_ROI
all_ROI$selected <- FALSE

In [7]:
# for a given bin, check if the bin falls into any number or ROI ranges and if it does, return the
# the index. After, drop the non-unique indices and finally compile the ROI list.

In [8]:
# only_spec is a vector of the filtered spectral bins that met Bonferroni significance and large
# and medium cohen's D effect size.
only_spec <- data[1]
nrow(only_spec)
head(only_spec)

[1] 127

,Spectral_Bin
,<chr>
76,8.48992060002728
89,8.22992060002728
106,7.88992060002728
110,7.80992060002728
117,7.66992060002728
118,7.64992060002728


In [9]:
rows <- c()
for (k in 1:nrow(all_ROI)){
    left_edge <- all_ROI[k,1]
    if (is.na(left_edge)){
        next}
    right_edge <- all_ROI[k,2]
    if (is.na(right_edge)){
        next}
    for (i in 1:nrow(only_spec)){
        bin_position <- (data[i,1])
        if (left_edge>bin_position & bin_position>right_edge){
            all_ROI$selected[k] <- TRUE
        }
    }
    #all_ROI["ROI_to_add"] <- rows
}

In [10]:
# sanity check
#all_ROI$selected

In [11]:
ROI_indices <- which(all_ROI$selected)

In [12]:
filtered_ROI <- filtered_ROI[ROI_indices,]

In [13]:
print(paste(data_set_root_dir, ":number of candidate compounds used for the peak-fitting search:",
            nrow(filtered_ROI)))

[1] "MTBLS424 :number of candidate compounds used for the peak-fitting search: 1853"


In [14]:
filtered_ROI_path <- paste(setwd_path, data_set_root_dir, '/output/', data_set_root_dir,
                       '_filtered_ROI.csv', sep='')
write.csv(filtered_ROI, file= filtered_ROI_path, row.names=FALSE)

In [15]:
huna_init <- rbind(huna_init, data.frame(type = "filtered_ROI_path",
                                         path = filtered_ROI_path))
huna_init <- rbind(huna_init, data.frame(type = "sample_type",
                                         path = sample_type))

save <- (paste(setwd_path,'/',data_set_root_dir, '/', data_set_root_dir, '_init.csv', sep=''))
write.csv(huna_init, file= save, row.names=FALSE)